In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os

# Set the path for current and output directories
current_dir = "/content/drive/MyDrive"
directory_name = "Convert Max Documentation"
output_path = os.path.join(current_dir, directory_name)

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"'{output_path}' folder created successfully.")

Mounted at /content/drive
'/content/drive/MyDrive/Convert Max Documentation' folder created successfully.


In [ ]:
! pip install requests beautifulsoup4 pdfkit
! pip install pdfkit
! apt-get install -y wkhtmltopdf

In [ ]:
import requests
import pdfkit
from bs4 import BeautifulSoup
import re
import time

# Set the URL and output file path
url = "https://docs.cycling74.com/max8/tutorials/00_mspindex"

# Define regular expressions (regex)
regex_pattern_main = re.compile(r'<h2[\s\S]*?>[\s\S]*?<ul>[\s\S]*?</h2>[\s\S]*?</ul>')
regex_pattern_nested = re.compile(r'<tr class="seealso-item">[\s\S]*?<td class="name">[\s\S]*?<span class="file-kind-icon" type="reference">[\s\S]*?</span>[\s\S]*?</td>')

# Configure wkhtmltopdf options
options = {
    "cookie-jar": "/tmp/cookies.txt",  # Exclude cookies
}

# Get initial URL and soup
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Set to keep track of processed href links
processed_links = set()

# Function to find href tags
def get_href_matches(regex_pattern, soup):
    matches = re.findall(regex_pattern, str(soup))
    href_pattern = r'<a href="([^"]+)">'
    href_name = r'>([^<]+)</a'
    for match in matches:
        href_matches = re.findall(href_pattern, match)
        name_matches = re.findall(href_name, match)
        for href_match, name_match in zip(href_matches, name_matches):
            file_name = name_match
            # processed_links.add(f"https://docs.cycling74.com{href_match}")
            yield href_match, file_name

# Function to make pdf files
def convert_webpage_to_pdf(url, filename):
    filename += ".pdf"
    output_file = os.path.join(output_path, filename)
    try:
        # Convert the webpage to PDF
        pdfkit.from_url(url, output_file, options=options, )
        print(f"PDF created: {output_file}")
        time.sleep(2) # Delay to avoid rapid request issues
    except Exception as e:
        print(f"Error creating {filename} for {url}: {str(e)}")
        with open('error_urls.txt', 'a') as f:
            f.write(f"{url},{filename}\n")

# Function to recursively extract href links and corresponding names
def extract_hrefs(main_soup, regex_pattern_main, regex_pattern_nested):
        stack = []
        for main_href, file_name in get_href_matches(regex_pattern_main, main_soup):
            main_url = f"https://docs.cycling74.com{main_href}"
            if main_url not in processed_links:
                stack.append(main_url) # Add the main_url to the stack if it was not processed yet
                convert_webpage_to_pdf(main_url, file_name)
                processed_links.add(main_url)
            while stack:
                nested_url = stack.pop()
                nested_response = requests.get(nested_url)
                nested_soup = BeautifulSoup(nested_response.text, "html.parser")
                for nested_href, file_name in get_href_matches(regex_pattern_nested, nested_soup):
                    # print(main_url)
                    # print(nested_url)
                    nested_url = f"https://docs.cycling74.com{nested_href}"
                    if nested_url not in (processed_links):
                        stack.append(nested_url) # Add the nested_url to the stack if it was not processed yet
                        convert_webpage_to_pdf(nested_url, file_name)
                        processed_links.add(nested_url)

extract_hrefs(soup, regex_pattern_main, regex_pattern_nested)

# Read the error_urls.txt file and retry converting the webpages to PDFs
with open('error_urls.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        url, filename = line.strip().split(',')
        convert_webpage_to_pdf(url, filename)

print("All PDFs created successfully.")

PDF created: /content/drive/MyDrive/Convert Max Documentation/Introduction.pdf
PDF created: /content/drive/MyDrive/Convert Max Documentation/How Digital Audio Works.pdf
PDF created: /content/drive/MyDrive/Convert Max Documentation/How MSP Works - Max Patches and the MSP Signal Network.pdf
Error creating Audio I/O - Audio input and output with MSP.pdf for https://docs.cycling74.com/max8/tutorials/04_mspaudioio: Command failed: /usr/bin/wkhtmltopdf --cookie-jar /tmp/cookies.txt --quiet https://docs.cycling74.com/max8/tutorials/04_mspaudioio /content/drive/MyDrive/Convert Max Documentation/Audio I/O - Audio input and output with MSP.pdf
Check whhtmltopdf output without 'quiet' option
[Errno 2] No such file or directory: '/content/drive/MyDrive/Convert Max Documentation/Audio I/O - Audio input and output with MSP.pdf' 
PDF created: /content/drive/MyDrive/Convert Max Documentation/adc~.pdf
PDF created: /content/drive/MyDrive/Convert Max Documentation/adstatus.pdf
PDF created: /content/drive